In [40]:
import os
import pandas as pd
import importlib
import json
import math
import random
import copy
from tqdm import tqdm
from itertools import accumulate


In [41]:

import csv
from datetime import datetime

# 定义一个函数来读取文件内容并返回所需的字典结构
def process_dyna_file(filename):
    data = {}

    with open(filename, 'r') as f:
        reader = csv.DictReader(f)
        for row in reader:
            entity_id = int(row['entity_id'])
            traj_id = int(row['traj_id'])
            coordinates = eval(row['coordinates'])  # 将字符串转为列表
            time_str = row['time']
            time_obj = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%SZ")

            # 若entity_id不在data字典中，初始化该entity_id的数据结构
            if entity_id not in data:
                data[entity_id] = {}
            
            # 若traj_id不在data[entity_id]中，初始化该traj_id的数据结构
            if traj_id not in data[entity_id]:
                start_time = time_obj  # 设置轨迹的起始时间
                data[entity_id][traj_id] = [[], [], [], [traj_id], [start_time], [entity_id], [entity_id]]
            
            # 添加数据
            data[entity_id][traj_id][0].append(coordinates[0])
            data[entity_id][traj_id][1].append(coordinates[1])
            time_gap = (time_obj - start_time).total_seconds()
            data[entity_id][traj_id][2].append(time_gap)
    
    return data

# 使用该函数
filename = "./raw/Chengdu_Taxi_Sample1_10_u50.dyna"
# filename = "./raw/Chengdu_Taxi_Sample1_10_u114.dyna"
result_data = process_dyna_file(filename)
# print(result_data)




In [28]:
list(result_data.keys())

[483,
 624,
 685,
 705,
 851,
 931,
 969,
 1240,
 1400,
 1821,
 1965,
 2328,
 2965,
 3262,
 4130,
 4368,
 4441,
 4487,
 4641,
 5527,
 5534,
 5763,
 5932,
 6266,
 6653,
 6931,
 6962,
 7114,
 8027,
 8223,
 8277,
 8604,
 9136,
 9236,
 9475,
 9531,
 9833,
 10177,
 10205,
 10589,
 10677,
 10687,
 11187,
 11264,
 11270,
 11755,
 12376,
 12730,
 13094,
 13759]

In [29]:
list(result_data[851].keys())

[49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]

In [30]:
result_data[851][49][0] # longitude

[104.066511,
 104.066304,
 104.066307,
 104.066111,
 104.065794,
 104.061141,
 104.055818,
 104.05148,
 104.045999,
 104.042489,
 104.039458,
 104.036072,
 104.032973,
 104.029105,
 104.025408]

In [31]:
result_data[851][49]

[[104.066511,
  104.066304,
  104.066307,
  104.066111,
  104.065794,
  104.061141,
  104.055818,
  104.05148,
  104.045999,
  104.042489,
  104.039458,
  104.036072,
  104.032973,
  104.029105,
  104.025408],
 [30.698732,
  30.693871,
  30.689341,
  30.686289,
  30.685954,
  30.685978,
  30.685258,
  30.6843,
  30.683065,
  30.684961,
  30.688613,
  30.69219,
  30.694904,
  30.698325,
  30.701535],
 [0.0,
  101.0,
  150.0,
  261.0,
  277.0,
  309.0,
  420.0,
  451.0,
  491.0,
  531.0,
  560.0,
  600.0,
  631.0,
  661.0,
  712.0],
 [49],
 [datetime.datetime(2014, 8, 21, 7, 24)],
 [851],
 [851]]

In [7]:
result_data[851][49][4]

[datetime.datetime(2014, 8, 21, 7, 24)]

In [8]:
result_data[851][49][5]

[851]

In [32]:
for key in result_data.keys():
    print(key)
    for k, v in result_data[key].items():
        v[-1][0] = 123

483
624
685
705
851
931
969
1240
1400
1821
1965
2328
2965
3262
4130
4368
4441
4487
4641
5527
5534
5763
5932
6266
6653
6931
6962
7114
8027
8223
8277
8604
9136
9236
9475
9531
9833
10177
10205
10589
10677
10687
11187
11264
11270
11755
12376
12730
13094
13759


In [33]:
result_data[851][49]

[[104.066511,
  104.066304,
  104.066307,
  104.066111,
  104.065794,
  104.061141,
  104.055818,
  104.05148,
  104.045999,
  104.042489,
  104.039458,
  104.036072,
  104.032973,
  104.029105,
  104.025408],
 [30.698732,
  30.693871,
  30.689341,
  30.686289,
  30.685954,
  30.685978,
  30.685258,
  30.6843,
  30.683065,
  30.684961,
  30.688613,
  30.69219,
  30.694904,
  30.698325,
  30.701535],
 [0.0,
  101.0,
  150.0,
  261.0,
  277.0,
  309.0,
  420.0,
  451.0,
  491.0,
  531.0,
  560.0,
  600.0,
  631.0,
  661.0,
  712.0],
 [49],
 [datetime.datetime(2014, 8, 21, 7, 24)],
 [851],
 [123]]

In [38]:
from datetime import datetime, timedelta
import csv
from math import radians, sin, cos, asin, sqrt

def geo_distance(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points
    on the earth (specified in decimal degrees)
    """
    lon1, lat1, lon2, lat2 = list(map(radians, list(map(float, [lon1, lat1, lon2, lat2]))))
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    r = 6371
    return c * r



def save_as_dyna1(modified_data, output_filename):
    ori_label_dict = {}
    atk_label_dict = {}
    with open(output_filename, 'w', newline='') as f:
        writer = csv.writer(f)
        # 写入标题行
        writer.writerow(["dyna_id", "type", "time", "entity_id", "traj_id", "coordinates", "current_dis", "current_state", "atk_id"])

        dyna_id = 0
        for entity_id, entity_data in modified_data.items():
            for traj_data in entity_data.values():
                longitudes, latitudes, time_gaps, traj_ids, start_times, ori_id, new_uid = traj_data
                ori_label_dict[traj_ids[0]] = ori_id[0]
                atk_label_dict[traj_ids[0]] = new_uid[0]
                prev_lat, prev_lon = latitudes[0], longitudes[0]
                current_dis_total = 0.0  # 初始化为0
                
                for i, (lat, lon, time_gap) in enumerate(zip(latitudes, longitudes, time_gaps)):
                    if i == 0:
                        current_dis = 0.0
                    else:
                        current_dis = geo_distance(prev_lon, prev_lat, lon, lat)
                        current_dis_total += current_dis  # 累计距离
                    
                    time_str = start_times[0] + timedelta(seconds=time_gap)
                    time_str = time_str.strftime("%Y-%m-%dT%H:%M:%SZ")
                    writer.writerow([dyna_id, "trajectory", time_str, ori_id[0], traj_ids[0], f"[{lon},{lat}]", current_dis_total, 1.0, new_uid[0]])
                    dyna_id += 1
                    prev_lat, prev_lon = lat, lon  # 更新上一个点的经纬度
    return ori_label_dict, atk_label_dict


In [39]:
output_filename = "./raw/Modified_Chengdu_Taxi_Sample.dyna"
save_as_dyna1(result_data, output_filename)

({0: 483,
  1: 483,
  2: 483,
  3: 483,
  4: 483,
  5: 483,
  6: 483,
  7: 483,
  8: 483,
  9: 483,
  10: 483,
  11: 483,
  12: 483,
  13: 483,
  14: 624,
  15: 624,
  16: 624,
  17: 624,
  18: 624,
  19: 624,
  20: 624,
  21: 624,
  22: 624,
  23: 624,
  24: 624,
  25: 624,
  26: 624,
  27: 685,
  28: 685,
  29: 685,
  30: 685,
  31: 685,
  32: 685,
  33: 685,
  34: 685,
  35: 685,
  36: 685,
  37: 685,
  38: 705,
  39: 705,
  40: 705,
  41: 705,
  42: 705,
  43: 705,
  44: 705,
  45: 705,
  46: 705,
  47: 705,
  48: 705,
  49: 851,
  50: 851,
  51: 851,
  52: 851,
  53: 851,
  54: 851,
  55: 851,
  56: 851,
  57: 851,
  58: 851,
  59: 851,
  60: 851,
  61: 931,
  62: 931,
  63: 931,
  64: 931,
  65: 931,
  66: 931,
  67: 931,
  68: 931,
  69: 931,
  70: 931,
  71: 931,
  72: 969,
  73: 969,
  74: 969,
  75: 969,
  76: 969,
  77: 969,
  78: 969,
  79: 969,
  80: 969,
  81: 969,
  82: 969,
  83: 969,
  84: 969,
  85: 1240,
  86: 1240,
  87: 1240,
  88: 1240,
  89: 1240,
  90: 1240,
  9

In [11]:
result_data[851]

{49: [[104.066511,
   104.066304,
   104.066307,
   104.066111,
   104.065794,
   104.061141,
   104.055818,
   104.05148,
   104.045999,
   104.042489,
   104.039458,
   104.036072,
   104.032973,
   104.029105,
   104.025408],
  [30.698732,
   30.693871,
   30.689341,
   30.686289,
   30.685954,
   30.685978,
   30.685258,
   30.6843,
   30.683065,
   30.684961,
   30.688613,
   30.69219,
   30.694904,
   30.698325,
   30.701535],
  [0.0,
   101.0,
   150.0,
   261.0,
   277.0,
   309.0,
   420.0,
   451.0,
   491.0,
   531.0,
   560.0,
   600.0,
   631.0,
   661.0,
   712.0],
  [49],
  [datetime.datetime(2014, 8, 21, 7, 24)],
  [851]],
 50: [[104.047867,
   104.050734,
   104.053821,
   104.055325,
   104.056719,
   104.058449,
   104.060354,
   104.061955,
   104.063064,
   104.062174,
   104.061657,
   104.061171,
   104.061939,
   104.062098,
   104.063848,
   104.06558,
   104.066549,
   104.066361,
   104.066337,
   104.066315,
   104.066299,
   104.066111,
   104.065794,
   10

In [12]:
def data2dic(data):
    result = {}
    for key, value in data.items():
        # 提取内部字典的值并存放到一个数组里
        inner_list = [v for k, v in value.items()]
        result[key] = inner_list
    return result

In [13]:
len(data2dic(result_data)[483])

14

In [42]:
import datetime

def original_to_new_format(original_data):
    new_format_data = {}
    for key, inner_dict in original_data.items():
        new_format_data[key] = []
        for inner_key, array in inner_dict.items():
            new_format_data[key].append(array)
    return new_format_data

def new_to_original_format(new_format_data):
    original_format_data = {}
    for key, array in new_format_data.items():
        # original_format_data[key] = {}
        for inner_array in array:
            if inner_array[5][0] not in original_format_data:
                original_format_data[inner_array[5][0]] = {}
            original_format_data[inner_array[5][0]][inner_array[3][0]] = inner_array
    return original_format_data


In [43]:
new1 = original_to_new_format(result_data)
new2 = new_to_original_format(new1)

In [45]:
new1[0]

[[[104.091825,
   104.095655,
   104.095301,
   104.092925,
   104.091664,
   104.087139,
   104.082508,
   104.081442,
   104.078464,
   104.077847,
   104.075899,
   104.071891,
   104.067715,
   104.063868,
   104.060837,
   104.056039,
   104.053661,
   104.052243,
   104.048519,
   104.043688,
   104.042384,
   104.042432,
   104.045408,
   104.0467,
   104.046688,
   104.046178,
   104.050109,
   104.055558],
  [30.682075,
   30.68019,
   30.677827,
   30.676658,
   30.673812,
   30.675871,
   30.678111,
   30.678695,
   30.67838,
   30.677232,
   30.676296,
   30.677549,
   30.680244,
   30.681473,
   30.68136,
   30.67954,
   30.680339,
   30.684501,
   30.683818,
   30.683964,
   30.685377,
   30.685524,
   30.684826,
   30.684114,
   30.683425,
   30.683181,
   30.683934,
   30.685099],
  [0.0,
   146.0,
   192.0,
   227.0,
   260.0,
   352.0,
   475.0,
   505.0,
   693.0,
   734.0,
   793.0,
   886.0,
   989.0,
   1020.0,
   1050.0,
   1139.0,
   1189.0,
   1267.0,
   1298.0

In [22]:
new1[851][0][-1] = [483]
new2 = new_to_original_format(new1)

In [23]:
new2

{483: {0: [[104.03098,
    104.034199,
    104.037237,
    104.039893,
    104.042708,
    104.046021,
    104.049346,
    104.052439,
    104.05602,
    104.058709,
    104.061536,
    104.064854,
    104.068625,
    104.071957,
    104.074842,
    104.078006,
    104.08111,
    104.084626,
    104.085195,
    104.085182,
    104.085652,
    104.085956,
    104.087944,
    104.090473,
    104.088561,
    104.088176,
    104.087429,
    104.087405,
    104.087368,
    104.088231,
    104.090684,
    104.092988,
    104.093565,
    104.094741,
    104.095939,
    104.097417,
    104.099138,
    104.100618,
    104.102374,
    104.101616,
    104.100685,
    104.099662],
   [30.66859,
    30.668072,
    30.667569,
    30.666459,
    30.664974,
    30.663309,
    30.661716,
    30.660749,
    30.660339,
    30.657964,
    30.656333,
    30.656507,
    30.656528,
    30.655399,
    30.65403,
    30.652509,
    30.650958,
    30.649271,
    30.646439,
    30.64643,
    30.646141,
    30.645

In [25]:
new2[483]

{0: [[104.03098,
   104.034199,
   104.037237,
   104.039893,
   104.042708,
   104.046021,
   104.049346,
   104.052439,
   104.05602,
   104.058709,
   104.061536,
   104.064854,
   104.068625,
   104.071957,
   104.074842,
   104.078006,
   104.08111,
   104.084626,
   104.085195,
   104.085182,
   104.085652,
   104.085956,
   104.087944,
   104.090473,
   104.088561,
   104.088176,
   104.087429,
   104.087405,
   104.087368,
   104.088231,
   104.090684,
   104.092988,
   104.093565,
   104.094741,
   104.095939,
   104.097417,
   104.099138,
   104.100618,
   104.102374,
   104.101616,
   104.100685,
   104.099662],
  [30.66859,
   30.668072,
   30.667569,
   30.666459,
   30.664974,
   30.663309,
   30.661716,
   30.660749,
   30.660339,
   30.657964,
   30.656333,
   30.656507,
   30.656528,
   30.655399,
   30.65403,
   30.652509,
   30.650958,
   30.649271,
   30.646439,
   30.64643,
   30.646141,
   30.645783,
   30.646979,
   30.646761,
   30.639897,
   30.638987,
   30.63

In [24]:
len(new1)

50

In [19]:
sum([len(v) for v in new1.values()])

631